# Linear auto-regressive model

A linear model that only looks at the recent forecasts to make a prediction on the current one.
It needs a new dataloader which outputs a window of forecasts instead of only one forecast.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import hydra
import numpy as np
import pandas as pd
import pathlib
import torch
import torch.nn as nn

import xarray as xr

In [ ]:
from smc01.postprocessing.dataset import SMCTimeSeriesDataset
from smc01.postprocessing.transform import CompositeTransform, DataframeToXarray, DataframeToExample

In [ ]:
with hydra.initialize_config_module('smc01.postprocessing.conf'):
    cfg = hydra.compose('validate', [
        'experiment/dataset=gdps_metar_timeseries',
        'experiment.dataset.begin=2019-02-01',
        'experiment.dataset.end=2020-01-01',
        'experiment.dataset.max_window_size=30',
    ])

In [ ]:
d = hydra.utils.instantiate(cfg.experiment.dataset)

In [ ]:
d[0].keys()

In [ ]:
class LinearTimeSeriesModel(nn.Module):
    def __init__(self, n_stations, n_steps, window_size, n_features):
        super().__init__()
        self.weights = nn.Parameter(torch.zeros(window_size, n_stations, n_steps, n_features))
        self.bias = nn.Parameter(torch.zeros(n_stations, n_steps))
        
    def forward(self, example):
        features = torch.nan_to_num(example['features'])
        
        linear = (features * self.weights).mean(dim=[0, -1])
                
        prediction = torch.where(
            ~example['forecast'].isnan(),
            example['forecast'] + linear - self.bias,
            0.0
        )
                
        return prediction


In [ ]:
loader = torch.utils.data.DataLoader(d, num_workers=4, batch_size=1)

In [ ]:
m = LinearTimeSeriesModel(1226, 81, 28, 1).cuda()
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [ ]:
b = next(iter(loader))

In [ ]:
for b in loader:
    b['forecast'] = b['forecast'].cuda()
    b['obs'] = b['obs'].cuda()
    b['features'] = b['features'].cuda()
    
    prediction = m(b)
    error = torch.where(~b['forecast'].isnan(), torch.square(prediction - b['obs']), 0.0)
    error = error.mean()
    error.backward()
    optimizer.step()
    print(b['forecast_date'], error)

In [ ]:
b['past_obs_2t'].shape

In [ ]:
b.keys()

In [ ]:
b['features'].shape

In [ ]:
m.weights.max()

In [ ]:
m.weights.min()

In [ ]:
m.bias.min()

In [ ]:
m.bias.max()

In [ ]:
m.bias.std()

In [ ]:
m.bias.mean()

In [ ]:
m.bias.mean(dim=1)

In [ ]:
m.weights.mean(dim=[0, 1])

In [ ]:
m.bias.mean(dim=0)